# Import Library

In [27]:
import numpy as np
import pandas as pd
from itertools import product


# DATA LOAD

In [15]:
df = pd.read_csv("../data/for_simulation.csv")

In [16]:

# 신뢰도 수준에 따른 리워드 금액과 예상 구매율 증가율 설정
reward_amounts = {
    'High': 15,
    'Mid': 10,
    'Low': 5,
    'Very Low': 0
}
increase_rates = {
    'High': 0.1,     # 10% 증가
    'Mid': 0.05,     # 5% 증가
    'Low': 0.02,     # 2% 증가
    'Very Low': 0.0  # 0% 증가
}

# 리워드 제공 전 총 거래액 및 평균 구매 금액 계산
initial_total_revenue = df['payment_value'].sum()
initial_avg_order = df['payment_value'].mean()

# 시뮬레이션 결과 저장
simulation_results = []

for trust_level, reward in reward_amounts.items():
    # 해당 신뢰도 수준에 맞는 구매 증가율 적용
    rate = increase_rates[trust_level]
    filtered_df = df[df['trust_level'] == trust_level]
    
    # 구매율 증가율 적용
    increased_orders = filtered_df['payment_value'] * (1 + rate)
    total_revenue_with_reward = increased_orders.sum()
    
    # 예상 추가 거래액과 기대 수익 계산
    additional_revenue = total_revenue_with_reward - initial_total_revenue
    expected_profit = total_revenue_with_reward * 0.1 - (reward * len(filtered_df))
    
    simulation_results.append({
        'Trust Level': trust_level,
        'Reward Amount': reward,
        'Increase Rate (%)': rate * 100,
        'Initial Total Revenue': initial_total_revenue,
        'Total Revenue with Reward': total_revenue_with_reward,
        'Additional Revenue': additional_revenue,
        'Expected Profit': expected_profit
    })

# 시뮬레이션 결과 DataFrame 생성 및 출력
simulation_df = pd.DataFrame(simulation_results)
print("신뢰도 수준에 따른 차등 리워드 지급 시 구매율 증가 시뮬레이션 결과:")
print(simulation_df)


신뢰도 수준에 따른 차등 리워드 지급 시 구매율 증가 시뮬레이션 결과:
  Trust Level  Reward Amount  Increase Rate (%)  Initial Total Revenue  \
0        High             15               10.0            19930492.56   
1         Mid             10                5.0            19930492.56   
2         Low              5                2.0            19930492.56   
3    Very Low              0                0.0            19930492.56   

   Total Revenue with Reward  Additional Revenue  Expected Profit  
0               3.460631e+05       -1.958443e+07       8431.31240  
1               1.368951e+07       -6.240987e+06     571880.50635  
2               4.245683e+06       -1.568481e+07     316478.33226  
3               2.415831e+06       -1.751466e+07     241583.12200  


In [17]:
# 신뢰도 수준에 따른 리워드 금액과 예상 구매율 증가율 설정
reward_amounts = {
    'High': 15,
    'Mid': 10,
    'Low': 5,
    'Very Low': 0
}
increase_rates = {
    'High': 0.1,     # 10% 증가
    'Mid': 0.05,     # 5% 증가
    'Low': 0.02,     # 2% 증가
    'Very Low': 0.0  # 0% 증가
}

# Trust level별 initial_total_revenue 계산
initial_revenues = df.groupby('trust_level')['payment_value'].sum().to_dict()

# 리워드 제공 전 총 거래액 계산
initial_total_revenue_all = df['payment_value'].sum()

# 시뮬레이션 결과 저장
simulation_results = []
total_revenue_with_simulation = 0  # 시뮬레이션 후 총 수익 계산

for trust_level, reward in reward_amounts.items():
    # Trust level별 초기 총 거래액
    initial_revenue = initial_revenues[trust_level]
    
    # Trust level별로 구매 증가율 적용
    rate = increase_rates[trust_level]
    filtered_df = df[df['trust_level'] == trust_level]
    
    # 예상 구매 증가율 적용
    increased_orders = filtered_df['payment_value'] * (1 + rate)
    total_revenue_with_reward = increased_orders.sum()
    
    # 시뮬레이션 후 총 수익에 반영
    total_revenue_with_simulation += total_revenue_with_reward
    
    # 예상 추가 거래액과 기대 수익 계산
    additional_revenue = total_revenue_with_reward - initial_revenue
    expected_profit = total_revenue_with_reward * 0.1 - (reward * len(filtered_df))
    
    simulation_results.append({
        'Trust Level': trust_level,
        'Reward Amount': reward,
        'Increase Rate (%)': rate * 100,
        'Initial Revenue': initial_revenue,
        'Total Revenue with Reward': total_revenue_with_reward,
        'Additional Revenue': additional_revenue,
        'Expected Profit': expected_profit
    })

# 시뮬레이션 결과 DataFrame 생성
simulation_df = pd.DataFrame(simulation_results)

# 시뮬레이션 전후 총 수익 비교
initial_total_profit = initial_total_revenue_all * 0.1
total_profit_with_simulation = total_revenue_with_simulation * 0.1 - sum(reward_amounts[trust_level] * len(df[df['trust_level'] == trust_level]) for trust_level in reward_amounts)

# 시뮬레이션 결과 출력
print("신뢰도 수준에 따른 차등 리워드 지급 시 시뮬레이션 결과:")
print(simulation_df)

print("\n시뮬레이션 전후 총 수익 비교:")
print(f"시뮬레이션 전 총 수익: {initial_total_profit:.2f}")
print(f"시뮬레이션 후 총 수익: {total_profit_with_simulation:.2f}")
print(f"총 수익 증가: {total_profit_with_simulation - initial_total_profit:.2f}")


신뢰도 수준에 따른 차등 리워드 지급 시 시뮬레이션 결과:
  Trust Level  Reward Amount  Increase Rate (%)  Initial Revenue  \
0        High             15               10.0        314602.84   
1         Mid             10                5.0      13037623.87   
2         Low              5                2.0       4162434.63   
3    Very Low              0                0.0       2415831.22   

   Total Revenue with Reward  Additional Revenue  Expected Profit  
0               3.460631e+05          31460.2840       8431.31240  
1               1.368951e+07         651881.1935     571880.50635  
2               4.245683e+06          83248.6926     316478.33226  
3               2.415831e+06              0.0000     241583.12200  

시뮬레이션 전후 총 수익 비교:
시뮬레이션 전 총 수익: 1993049.26
시뮬레이션 후 총 수익: 1138373.27
총 수익 증가: -854675.98


In [23]:
# 신뢰도 수준에 따른 리워드 금액과 예상 구매율 증가율 설정
reward_amounts = {
    'High': 10,
    'Mid': 6,
    'Low': 3,
    'Very Low': 0
}
increase_rates = {
    'High': 0.1,     
    'Mid': 0.1,     
    'Low': 0.1,     
    'Very Low': 0.1  
}

# Trust level별 초기 총 수익(판매 대금의 10%) 계산
initial_revenues = {level: df[df['trust_level'] == level]['payment_value'].sum() * 0.1 for level in df['trust_level'].unique()}

# 시뮬레이션 결과 저장
simulation_results = []
total_revenue_with_simulation = 0  # 시뮬레이션 후 총 수익 계산

for trust_level, reward in reward_amounts.items():
    # Trust level별 초기 총 수익
    initial_revenue = initial_revenues[trust_level]
    
    # Trust level별 리워드 제공액 계산 (리워드 금액 * 해당 수준의 리뷰 수)
    reward_provided = reward * len(df[df['trust_level'] == trust_level])
    
    # 구매 증가율에 따른 수익 상승액 계산 (1 + rate를 곱함)
    rate = increase_rates[trust_level]
    total_revenue_with_reward = initial_revenue * (1 + rate)
    
    # 예상 추가 수익 계산
    additional_revenue = total_revenue_with_reward - initial_revenue
    
    # 기대 수익 계산: 총 수익에서 리워드 제공액을 차감
    expected_profit = additional_revenue - reward_provided
    
    # 시뮬레이션 결과 저장
    simulation_results.append({
        'Trust Level': trust_level,
        'Reward Amount': reward,
        'Increase Rate (%)': rate * 100,
        'Initial Revenue': initial_revenue,
        'Total Revenue with Reward': total_revenue_with_reward,
        'Reward Provided': reward_provided,
        'Additional Revenue': additional_revenue,
        'Expected Profit': expected_profit
    })

# 시뮬레이션 결과 DataFrame 생성 및 출력
simulation_df = pd.DataFrame(simulation_results)
print("신뢰도 수준에 따른 차등 리워드 지급 시 시뮬레이션 결과:")
print(simulation_df)

# 시뮬레이션 전후 총 수익 비교
initial_total_profit = sum(initial_revenues.values())
total_profit_with_simulation = simulation_df['Expected Profit'].sum()

print("\n시뮬레이션 전후 총 수익 비교:")
print(f"시뮬레이션 전 총 수익: {initial_total_profit:.2f}")
print(f"시뮬레이션 후 총 수익: {total_profit_with_simulation:.2f}")
print(f"총 수익 증가: {total_profit_with_simulation - initial_total_profit:.2f}")


신뢰도 수준에 따른 차등 리워드 지급 시 시뮬레이션 결과:
  Trust Level  Reward Amount  Increase Rate (%)  Initial Revenue  \
0        High             10               10.0        31460.284   
1         Mid              6               10.0      1303762.387   
2         Low              3               10.0       416243.463   
3    Very Low              0               10.0       241583.122   

   Total Revenue with Reward  Reward Provided  Additional Revenue  \
0               3.460631e+04            17450           3146.0284   
1               1.434139e+06           478242         130376.2387   
2               4.578678e+05            64854          41624.3463   
3               2.657414e+05                0          24158.3122   

   Expected Profit  
0      -14303.9716  
1     -347865.7613  
2      -23229.6537  
3       24158.3122  

시뮬레이션 전후 총 수익 비교:
시뮬레이션 전 총 수익: 1993049.26
시뮬레이션 후 총 수익: -361241.07
총 수익 증가: -2354290.33


In [44]:

# 리워드 금액, 구매율 증가율, 신뢰도 상승 확률
reward_combinations = [
    {'High': 10, 'Mid': 5, 'Low': 3, 'Very Low': 0},
    {'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0},
    {'High': 5, 'Mid': 3, 'Low': 2, 'Very Low': 0},
    
    
    
    {'High': 15, 'Mid': 10, 'Low': 5, 'Very Low': 0},
    {'High': 20, 'Mid': 15, 'Low': 10, 'Very Low': 5},
    {'High': 25, 'Mid': 20, 'Low': 15, 'Very Low': 5}
]

increase_rate_combinations = [
    {'High': 0.1, 'Mid': 0.1, 'Low': 0.1, 'Very Low': 0.1},
    {'High': 0.2, 'Mid': 0.2, 'Low': 0.2, 'Very Low': 0.2},
    {'High': 0.15, 'Mid': 0.15, 'Low': 0.15, 'Very Low': 0.15},

    {'High': 0.1, 'Mid': 0.05, 'Low': 0.02, 'Very Low': 0.0},
    {'High': 0.15, 'Mid': 0.1, 'Low': 0.05, 'Very Low': 0.02},
    {'High': 0.2, 'Mid': 0.15, 'Low': 0.1, 'Very Low': 0.02}
]

# trust_level_increase_combinations = [
#     {'Very Low': 0.2, 'Low': 0.1, 'Mid': 0.05, 'High': 0.0},
#     {'Very Low': 0.3, 'Low': 0.2, 'Mid': 0.1, 'High': 0.0},
#     {'Very Low': 0.4, 'Low': 0.3, 'Mid': 0.2, 'High': 0.0}
# ]

# 모든 조합 생성
# all_combinations = list(product(reward_combinations, increase_rate_combinations, trust_level_increase_combinations))

all_combinations = list(product(reward_combinations, increase_rate_combinations))
# 시뮬레이션 결과 저장
simulation_results = []

# 모든 조합에 대해 시뮬레이션 실행
# for reward_amounts, increase_rates, trust_level_increase_prob in all_combinations:

for reward_amounts, increase_rates in all_combinations:
    # Trust level별 초기 총 수익(판매 대금의 10%) 계산
    initial_revenues = {level: int(df[df['trust_level'] == level]['payment_value'].sum() * 0.1) for level in df['trust_level'].unique()}
    initial_trust_distribution = df['trust_level'].value_counts(normalize=True).to_dict()
    
    # 신뢰도 분포 업데이트
    updated_trust_distribution = initial_trust_distribution.copy()
    # for trust_level, prob in trust_level_increase_prob.items():
    #     if trust_level == 'High':
    #         continue  # High는 최고 등급이므로 변화 없음
        
    #     next_level = {
    #         'Very Low': 'Low',
    #         'Low': 'Mid',
    #         'Mid': 'High'
    #     }[trust_level]
        
    #     # 신뢰도 이동
    #     updated_trust_distribution[next_level] = updated_trust_distribution.get(next_level, 0) + updated_trust_distribution[trust_level] * prob
    #     updated_trust_distribution[trust_level] *= (1 - prob)
    
    # 초기 수익 및 시뮬레이션 계산
    initial_total_revenue = sum(initial_revenues.values())
    reward_provided_total = 0
    simulation_total_revenue = 0

    for trust_level, reward in reward_amounts.items():
        # 리워드 제공액 계산
        trust_level_count = len(df[df['trust_level'] == trust_level])
        reward_provided = reward * trust_level_count
        reward_provided_total += reward_provided
        
        # 구매 증가율에 따른 시뮬레이션 수익
        initial_revenue = initial_revenues.get(trust_level, 0)
        rate = increase_rates[trust_level]
        simulation_revenue = int(initial_revenue * (1 + rate))
        simulation_total_revenue += simulation_revenue
    
    # 시뮬레이션 순수익
    simulation_net_profit = simulation_total_revenue - reward_provided_total
    
    # 리워드 제공에 따른 추가 수익
    additional_revenue = simulation_net_profit - initial_total_revenue
    
    # 시뮬레이션 결과 저장
    simulation_results.append({
        'Reward Combination': reward_amounts,
        'Increase Rate Combination': increase_rates,
        # 'Trust Level Increase Probabilities': trust_level_increase_prob,
        # 'Initial Trust Distribution': initial_trust_distribution,
        # 'Updated Trust Distribution': updated_trust_distribution,
        'Initial Revenue': initial_total_revenue,
        'Simulation Revenue': simulation_total_revenue,
        'Reward Provided': reward_provided_total,
        'Simulation Net Profit': simulation_net_profit,
        'Additional Revenue': additional_revenue
    })

# 결과 DataFrame 생성
simulation_df = pd.DataFrame(simulation_results)

# 상위 10개의 시뮬레이션 결과 출력
print("다양한 시뮬레이션 결과:")

print(simulation_df.nlargest(10, 'Simulation Net Profit'))


다양한 시뮬레이션 결과:
                                 Reward Combination  \
7    {'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}   
8    {'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}   
13   {'High': 5, 'Mid': 3, 'Low': 2, 'Very Low': 0}   
11   {'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}   
6    {'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}   
14   {'High': 5, 'Mid': 3, 'Low': 2, 'Very Low': 0}   
10   {'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}   
17   {'High': 5, 'Mid': 3, 'Low': 2, 'Very Low': 0}   
1   {'High': 10, 'Mid': 5, 'Low': 3, 'Very Low': 0}   
12   {'High': 5, 'Mid': 3, 'Low': 2, 'Very Low': 0}   

                            Increase Rate Combination  Initial Revenue  \
7   {'High': 0.2, 'Mid': 0.2, 'Low': 0.2, 'Very Lo...          1993048   
8   {'High': 0.15, 'Mid': 0.15, 'Low': 0.15, 'Very...          1993048   
13  {'High': 0.2, 'Mid': 0.2, 'Low': 0.2, 'Very Lo...          1993048   
11  {'High': 0.2, 'Mid': 0.15, 'Low': 0.1, 'Very L...          1993048   
6   {'High

In [45]:
simulation_df.sort_values(by=["Additional Revenue"],ascending=False)

,Reward Combination,Increase Rate Combination,Initial Revenue,Simulation Revenue,Reward Provided,Simulation Net Profit,Additional Revenue
7,"{'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}","{'High': 0.2, 'Mid': 0.2, 'Low': 0.2, 'Very Lo...",1993048,2391656,186267,2205389,212341
8,"{'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}","{'High': 0.15, 'Mid': 0.15, 'Low': 0.15, 'Very...",1993048,2292004,186267,2105737,112689
13,"{'High': 5, 'Mid': 3, 'Low': 2, 'Very Low': 0}","{'High': 0.2, 'Mid': 0.2, 'Low': 0.2, 'Very Lo...",1993048,2391656,291082,2100574,107526
11,"{'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}","{'High': 0.2, 'Mid': 0.15, 'Low': 0.1, 'Very L...",1993048,2241359,186267,2055092,62044
6,"{'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}","{'High': 0.1, 'Mid': 0.1, 'Low': 0.1, 'Very Lo...",1993048,2192352,186267,2006085,13037
14,"{'High': 5, 'Mid': 3, 'Low': 2, 'Very Low': 0}","{'High': 0.15, 'Mid': 0.15, 'Low': 0.15, 'Very...",1993048,2292004,291082,2000922,7874
10,"{'High': 3, 'Mid': 2, 'Low': 1, 'Very Low': 0}","{'High': 0.15, 'Mid': 0.1, 'Low': 0.05, 'Very ...",1993048,2153786,186267,1967519,-25529
17,"{'High': 5, 'Mid': 3, 'Low': 2, 'Very Low': 0}","{'High': 0.2, 'Mid': 0.15, 'Low': 0.1, 'Very L...",1993048,2241359,291082,1950277,-42771
1,"{'High': 10, 'Mid': 5, 'Low': 3, 'Very Low': 0}","{'High': 0.2, 'Mid': 0.2, 'Low': 0.2, 'Very Lo...",1993048,2391656,480839,1910817,-82231
12,"{'High': 5, 'Mid': 3, 'Low': 2, 'Very Low': 0}","{'High': 0.1, 'Mid': 0.1, 'Low': 0.1, 'Very Lo...",1993048,2192352,291082,1901270,-91778
